In [ ]:
import sys; sys.path.append('..')
from bending_validation import *
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, vis, numpy as np, time

r, direction = pickle.load(open('data/finite_transport_example.pkl', 'rb'))
dc = r.deformedConfiguration()
P = np.array(r.deformedPoints())

view = linkage_vis.LinkageViewer(r, width=800, height=600)
view.ghostColor = 'lightgray'
cview = linkage_vis.CenterlineViewer(r, width=800, height=600, superView=view)
view.avoidRedrawFlicker = cview.avoidRedrawFlicker = True
cview.showPoints()
def updateViews():
    dc = r.deformedConfiguration()
    view.update(transparent='true')
    cview.update(vectorField=np.array([f.d1 for f in dc.referenceDirectors]))
updateViews()
view.show()

In [ ]:
#cview.update(vectorField=np.array([f.d1 for f in dc.sourceReferenceDirectors]))
cview.update(vectorField=np.array(dc.sourceTangent))

In [ ]:
def randomUnitVector():
    while True:
        v = np.random.uniform(low=-1, high=1, size=3)
        n = np.linalg.norm(v)
        if (n > 1 or n < 1e-7): continue
        return v / n
rotation = scipy.spatial.transform.Rotation.from_rotvec
def rotateLastEdge(magnitude, direction = None):
    if (direction is None): direction = randomUnitVector()
    defP = P.copy()
    x1 = P[1, :]
    e = P[2, :] - x1
    defP[2, :] = x1 + rotation(magnitude * direction).apply(e)
    r.setDeformedConfiguration(defP, r.thetas())
    updateViews()

import time
while True:
    for alpha in np.linspace(0, 2 * np.pi, 100):
        rotateLastEdge(0.1 * np.sin(alpha), direction)
        time.sleep(0.02)

In [ ]:
vf = vis.fields.VectorField(cview.mesh, r.gradient()[0:r.thetaOffset()].reshape((-1, 3)), vmin=0, vmax=2.66025584e-05)
cview.update(vectorField=vf)

In [ ]:
g_finite_xport = r.gradient()
g_inf_xport = r.gradient(updatedSource=True)

In [ ]:
r.updateSourceFrame()

In [ ]:
r.gradient() - g_finite_xport

In [ ]:
r.gradient() - g_inf_xport